# A4. Loading sensortran files
This example loads sensortran files. Only single-ended measurements are currently supported.
Sensortran files are in binary format. The library requires the `*BinaryRawDTS.dat` and `*BinaryTemp.dat` files.

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
    
from dtscalibration import read_sensortran_files

The example data files are located in `./python-dts-calibration/tests/data`.

In [ ]:
filepath = os.path.join('..', '..', 'tests', 'data', 'sensortran_binary')
print(filepath)

In [ ]:
filepathlist = sorted(glob.glob(os.path.join(filepath, '*.dat')))
filenamelist = [os.path.basename(path) for path in filepathlist]

for fn in filenamelist:
    print(fn)

We will simply load in the binary files

In [ ]:
ds = read_sensortran_files(directory=filepath)

The object tries to gather as much metadata from the measurement files as possible (temporal and spatial coordinates, filenames, temperature probes measurements). All other configuration settings are loaded from the first files and stored as attributes of the `DataStore`. Sensortran's data files contain less information than the other manufacturer's devices, one being the acquisition time. The acquisition time is needed for estimating variances, and is set a constant 1s.

In [ ]:
print(ds)

The sensortran files differ from other manufacturers, in that they return the 'counts' of the Stokes and anti-Stokes signals. These are not corrected for offsets, which has to be done manually for proper calibration. 

Based on the data available in the binary files, the library estimates a zero-count to correct the signals, but this is not perfectly accurate or constant over time. For proper calibration, the offsets would have to be incorporated into the calibration routine.

In [ ]:
ds

In [ ]:
ds0 = ds.isel(time=0)

plt.figure()
ds0.st.plot(label='Stokes signal')
plt.axhline(ds0.st_zero.values, c='r', label="'zero' measurement")
plt.legend()
plt.title('')
plt.axhline(c='k')

After a correction and rescaling (for human readability) the data will look more like other manufacturer's devices

In [ ]:
ds['st'] = (ds.st - ds.st_zero)/1e4
ds['ast'] = (ds.ast - ds.ast_zero)/1e4

In [ ]:
ds.isel(time=0).st.plot(label='Stokes intensity')
ds.isel(time=0).ast.plot(label='anti-Stokes intensity')
plt.legend()
plt.axhline(c='k', lw=1)
plt.xlabel('')
plt.title('')
plt.ylim([-50,500])